In [5]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import Docx2txtLoader
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

import transformers
import peft
import torch

from tqdm.autonotebook import tqdm as notebook_tqdm

False
'CUDASetup' object has no attribute 'cuda_available'


/home/savelov/miniconda3/envs/llama2/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


## LLM

In [ ]:
model_id = "IlyaGusev/saiga2_13b_lora"

hf_auth = 'TOKEN'

model_config = peft.PeftConfig.from_pretrained(model_id)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_config.base_model_name_or_path,
    trust_remote_code=True,
    config=model_config,
    device_map='auto',
    use_auth_token=hf_auth
).eval()

model = peft.PeftModel.from_pretrained(
    model,
    model_id,
    torch_dtype=torch.float16
).eval()

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/home/savelov/miniconda3/envs/llama2/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/savelov/miniconda3/envs/llama2/lib/python3.11/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/savelov/miniconda3/envs/llama2/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

In [ ]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    temperature=0.1,
    do_sample=True,
    max_new_tokens=512,
    repetition_penalty=1.1,
    num_beams=2,
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartFo

## Embedding the dataset

In [6]:
embed_model_id = 'intfloat/multilingual-e5-base'

device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

In [7]:
raw_documents = Docx2txtLoader('Приказ Минтранса России от 23 06 2022 N 250.docx').load()
text_splitter = SentenceTransformersTokenTextSplitter(chunk_size=500, chunk_overlap=100)

documents = text_splitter.split_documents(raw_documents)

In [8]:
db = Chroma.from_documents(documents, embed_model)

In [9]:
retriever = db.as_retriever(search_type="mmr")

## RetrievalQA

In [16]:
llm = HuggingFacePipeline(pipeline=generate_text)

In [17]:
rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',
    retriever=retriever
)

In [19]:
rag_pipeline

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7fe8ec6c06d0>)), document_variable_name='context'), retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7fea1f041f90>, search_type='mmr'))

In [20]:
question_template = "system:\n"\
                    "Используй следующие фрагменты контекста, чтобы ответить на вопрос в конце. "\
                    "Если ты не знаешь ответа, просто скажи, что не знаешь что ответить, не пытайся придумать ответ."\
                    "\n\n{context}\n\nВопрос: {question}\nОтвет: "

rag_pipeline.combine_documents_chain.llm_chain.prompt.template = question_template

In [21]:
rag_pipeline

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='system:\nИспользуй следующие фрагменты контекста, чтобы ответить на вопрос в конце. Если ты не знаешь ответа, просто скажи, что не знаешь что ответить, не пытайся придумать ответ.\n\n{context}\n\nВопрос: {question}\nОтвет: '), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7fe8ec6c06d0>)), document_variable_name='context'), retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7fea1f041f90>, search_type='mmr'))

---

In [22]:
question = "Между кем должны согласовываться пересечения путей инженерными сооружениями?"

In [23]:
retriever.vectorstore.similarity_search(question, k=5)

[Document(page_content='##дерации, 2003, n 2, ст. 169 ). 61. пересечения железнодорожных путеи инженерными сооружениями, в том числе линиями связи, электропередачи, нефте -, газо -, продуктопроводами и другими наземными и подземными сооружениями, на стадии проектирования согласовываются владельцами указанных объектов с владельцем инфраструктуры ( владельцем железнодорожных путеи необщего пользования ). 62. запрещается примыкание проектируемых и вновь строящихся железнодорожных путеи необ', metadata={'source': 'Приказ Минтранса России от 23 06 2022 N 250.docx'}),
 Document(page_content='##освязи, станционных здании, сооружении ( за исключением искусственных сооружении ) и устроиств инфраструктуры на основании проектнои, ремонтнои или эксплуатационнои документации определяются локальным нормативным актом владельца инфраструктуры ( владельца железнодорожных путеи необщего пользования ). владелец инфраструктуры ( владелец железнодорожных путеи необщего пользования ) должен проводить не реж

In [24]:
rag_pipeline(question)

{'query': 'Между кем должны согласовываться пересечения путей инженерными сооружениями?',
 'result': ' Владельцы указанных объектов с владельцем инфраструктуры ( владельцем железнодорожных путеи необщего пользования ).'}

In [100]:
from functools import cache

In [101]:
message_template = "{role}\n{content}\n"

@cache
def answerUsingKnowledgeBase(question):
    answer = rag_pipeline(question)
    return answer['result'].strip()


def answerUsingHistory(history_json : list[dict]):
    history = ""
    
    for message in history_json:
        message_text = message_template.format(**message)
        history += message_text
            
    answer = llm(history)
    
    return answer.strip()

In [103]:
answerUsingKnowledgeBase('На ком ответственность за содержание путей?')

'1. Ответственность за содержание путей лежит на владельцах железнодорожных путей необщего пользования.\n2. Владельцы железнодорожных путей необщего пользования также ответственны за соблюдение периодичности выполнения планово-предупредительных ремонтов и за соблюдение назначенного срока службы (ресурса) железнодорожного подвижного состава.\n3. Владельцы железнодорожных станций или железнодорожных путей необщего пользования, в том числе на специально оборудованных площадках, также ответственны за содержание в исправном техническом состоянии железнодорожного подвижного состава.\n4. Машинист, при производстве работ - руководитель работ, назначаемый владельцем инфраструктуры ( владельцем железнодорожных путеи необщего пользования ), ответственны за безопасность движения и людьми, находящимися на СПК, при движении.\n5. Количество перевозимых в СПК людьми и характеристики груза не должны нарушать нормы, установленные владельцем инфраструктуры ( владельцем железнодорожных путеи необщего поль

In [41]:
history = [
    {'role': 'system', 'content': 'Используй следующие фрагменты контекста, чтобы ответить на вопрос в конце. Если ты не знаешь ответа, просто скажи, что не знаешь что ответить, не пытайся придумать ответ'},
    {'role': 'user', 'content': 'Что такое железная дорога?'},
    {'role': 'bot', 'content': 'Железная дорога - это система железнодорожных путей, станций и других объектов, используемая для перевозки пассажиров и грузов на железнодорожном транспорте.'},
    {'role': 'user', 'content': 'Объясни это так, как будто мне 5 лет'},
    {'role': 'bot', 'content': 'Железная дорога - это большая дорога, по которой ездят большие машины, которые называются поездами.'},
]

In [ ]:
answerUsingHistory(history)

### submit

In [85]:
import pandas as pd
from tqdm.auto import tqdm
import re

In [86]:
submit_df = pd.read_excel("Questions_list.xlsx")

In [98]:
submit_df["question"].unique().shape

(845,)

In [87]:
abbreviations = set()

for index, row in submit_df.iterrows():
    for abb in re.findall(r"\b[А-Я]{2,}\b", row['question']):
        abbreviations.add(abb)

abbreviations

{'ЖД', 'КВЗ', 'МВПС', 'ПТО', 'ПТЭ', 'РФ', 'ССПС'}

In [88]:
abbreviations_dict = {
    'ЖД': 'железнодорожный',
    'МВПС': 'моторвагонный подвижной состав',
    'ПТО': 'пункт технического обслуживания вагонов',
    'ПТЭ': 'правила технической эксплуатации',
    'ССПС': 'специальный самоходный подвижный состав'
}

In [89]:
def replace_abbreviations(question):
    for abbreviation, full_text in abbreviations_dict.items():
        question = question.replace(abbreviation, full_text)
    
    return question

submit_df['question'] = submit_df['question'].apply(replace_abbreviations)
submit_df['question']

0           Зачем нужны правила технической эксплуатации?
1       Кто должен выполнять правила технической экспл...
2       Где найти информацию о требованиях к видимым с...
3         Где найти правила приема и отправления поездов?
4       Какие обязанности у работников железнодорожный...
                              ...                        
1065    Какие требования предъявляются к расстоянию ви...
1066                  Как отмечаются неактивные семафоры?
1067    Как располагаются оповестительные щиты перед в...
1068    Что должно произойти с крылом семафора в случа...
1069    Чем устанавливается порядок освещения сигнальн...
Name: question, Length: 1070, dtype: object

In [118]:
answers = []

with open("checkpoint.txt", 'w') as f:
    for question in tqdm(submit_df["question"][292:]):
        answer = answerUsingKnowledgeBase(str(question))
        answers.append(answer)

        f.write(answer + '\n\n<sep>\n\n')

  0%|          | 0/778 [00:00<?, ?it/s]

/home/savelov/miniconda3/envs/llama2/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/savelov/miniconda3/envs/llama2/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/savelov/miniconda3/envs/llama2/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/savelov/miniconda3/envs/llama2/lib/python3.11/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/savelov/miniconda3/envs/ll

In [ ]:
submit_df["answer"] = answers